In [98]:
import pandas as pd
import numpy as np
import spacy

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.layers import TextVectorization, Embedding, Conv1D, MaxPooling1D, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping

import nltk
from nltk.corpus import stopwords

In [67]:
nltk.download('stopwords')
stop_words = set(stopwords.words('portuguese'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\janna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [68]:
df = pd.read_excel('../data/DATA_FULL_TRAIN.xlsx')

In [69]:
df.head()

,Texto,Classe
0,SERVICO E LICENCAS DE SOFTWARE DE USO DO CALL CEN,MACHINERY
1,UPGRADE DO SISTEMA DE SEGURANCA DO SITE ALEXANDRE,MACHINERY
2,UPGRADE DO SISTEMA DE SEGURANCA DO SITE ALEXANDRE,MACHINERY
3,REFRIGERADOR 342 L FF CLEAN,MACHINERY
4,REFRIGERADOR 342 L FF CLEAN,MACHINERY


In [70]:
nlp = spacy.load("pt_core_news_sm")

def preprocess_text(text, stop_words=[]):
    doc = nlp(text)
    lemmatized_words = [token.lemma_.lower() for token in doc if token.lemma_ not in stop_words and len(token.lemma_) > 1]
    processed_text = ' '.join(lemmatized_words)
    return processed_text


In [71]:
df['Texto'] = df['Texto'].apply(preprocess_text)

In [159]:
X = df['Texto']
y = df['Classe']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)

In [160]:
tokenizer = Tokenizer(
    num_words=None,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True,
    split=' ',
)

In [161]:
sequences = tokenizer.fit_on_texts(X_train)
print(tokenizer.word_index)

{'de': 1, 'cadeira': 2, 'em': 3, 'mca': 4, 'adm': 5, 'fixa': 6, 'mod': 7, 'c': 8, 'gb': 9, 'cco': 10, 'dell': 11, 'mesa': 12, 'thinkpad': 13, 'monitor': 14, 'giratória': 15, '1': 16, 'notebook': 17, 'com': 18, 'painel': 19, 'para': 20, 'aco': 21, 'gavetas': 22, 'ar': 23, 'microcomputador': 24, '64': 25, 'madeira': 26, 'portas': 27, 'i5': 28, 'serie': 29, 'infantil': 30, 'giratoria': 31, 'carteira': 32, 'escolar': 33, 'lenovo': 34, 'tipo': 35, 'poltrona': 36, 'iphone': 37, 'atendimento': 38, 'gaveteiro': 39, 'balcão': 40, 'armario': 41, 'lugares': 42, 'padrão': 43, '5': 44, 'inox': 45, 'standard': 46, 'vostro': 47, 'air': 48, '2': 49, 'formica': 50, 'linha': 51, 'black': 52, 'ipad': 53, '8350u': 54, 'licença': 55, 'hp': 56, 'caixa': 57, 'pa': 58, 'kit': 59, '14': 60, 'o': 61, 'executiva': 62, 'base': 63, 't480': 64, 'core': 65, 'instalação': 66, 'smartphone': 67, 'p': 68, '12': 69, 'longarina': 70, 'tecido': 71, 'sob': 72, 'eletrico': 73, 'estrutura': 74, 'condicionado': 75, 'medida': 7

In [162]:
vectorizer = TextVectorization(
    max_tokens=20000,
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    output_mode='int',
    output_sequence_length=100
)

vectorizer.adapt(X_train)

In [173]:
X_train = vectorizer(np.array([[s] for s in X_train])).numpy()
X_test = vectorizer(np.array([[s] for s in X_test])).numpy()

In [77]:
word_index = vectorizer.get_vocabulary()

In [78]:
vectorizer("SERVICO LICENCAS DE SOFTWARE DE USO DO CALL CEN")

<tf.Tensor: shape=(9,), dtype=int64, numpy=array([ 576, 3582,    2,  148,    2,  160,   82, 2222,    1], dtype=int64)>

In [84]:
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

num_labels = len(le.classes_)

In [190]:
model = Sequential([
    # Camada de Embedding para transformar números inteiros (índices de palavras) em vetores densos
    Embedding(input_dim=len(vectorizer.get_vocabulary()), output_dim=50, input_length=100),
    
    # Camada Conv1D para convolução unidimensional sobre a sequência de entrada
    Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'),
    
    # Camada MaxPooling1D para reduzir o tamanho da entrada pela extração do máximo da janela
    MaxPooling1D(pool_size=2),
    
    # Dropout para regularização, desativando aleatoriamente 50% dos neurônios durante o treinamento
    Dropout(0.5),
    
    # Camada Bidirectional LSTM para capturar dependências de sequência em ambas as direções
    Bidirectional(LSTM(units=64, dropout=0.2, recurrent_dropout=0.3, return_sequences=True)),
    
    # Outra camada Bidirectional LSTM, agora sem return_sequences, para processar a saída da primeira camada LSTM
    Bidirectional(LSTM(units=32)),
    
    # Dropout adicional para a segunda camada LSTM
    Dropout(0.2),
    
    # Camada Dense totalmente conectada com ativação 'relu' para aprendizado de características
    Dense(64, activation='relu'),
    
    # Camada Dense de saída com ativação 'softmax' para classificação multiclasse
    Dense(num_labels, activation='softmax')
])


In [193]:
# Compilar o modelo com otimizador 'adam' e função de perda 'sparse_categorical_crossentropy' para classificação multiclasse
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [194]:
history = model.fit(X_train, y_train_encoded, epochs=8, validation_split=0.2)

Epoch 1/8
1218/1218 [==============================] - 49s 36ms/step - loss: 0.4473 - accuracy: 0.8452 - val_loss: 0.2380 - val_accuracy: 0.9306
Epoch 2/8
1218/1218 [==============================] - 44s 36ms/step - loss: 0.1769 - accuracy: 0.9494 - val_loss: 0.1993 - val_accuracy: 0.9436
Epoch 3/8
1218/1218 [==============================] - 44s 37ms/step - loss: 0.1174 - accuracy: 0.9662 - val_loss: 0.1835 - val_accuracy: 0.9468
Epoch 4/8
1218/1218 [==============================] - 45s 37ms/step - loss: 0.0904 - accuracy: 0.9747 - val_loss: 0.1873 - val_accuracy: 0.9481
Epoch 5/8
1218/1218 [==============================] - 45s 37ms/step - loss: 0.0766 - accuracy: 0.9777 - val_loss: 0.1912 - val_accuracy: 0.9517
Epoch 6/8
1218/1218 [==============================] - 45s 37ms/step - loss: 0.0655 - accuracy: 0.9802 - val_loss: 0.2112 - val_accuracy: 0.9461
Epoch 7/8
1218/1218 [==============================] - 45s 37ms/step - loss: 0.0595 - accuracy: 0.9817 - val_loss: 0.2241 - val_ac

In [195]:
loss, accuracy = model.evaluate(X_test, y_test_encoded)
print(f'Acurácia: {accuracy * 100:.2f}%')

653/653 [==============================] - 5s 7ms/step - loss: 0.2451 - accuracy: 0.9447
Acurácia: 94.47%
